# Rastreo de insumos
## Para las 3 sucursales analizar la cantidad que se envio durante una semana de domingo a sabado, restando las ventas.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, date, time, timedelta
import calendar


### Descarga de datos y creacion de diccionario

In [76]:
Tdata = pd.ExcelFile('tepeyac.xlsx')
Pdata = pd.ExcelFile('palomar.xlsx')
Idata = pd.ExcelFile('inglaterra.xlsx')
Idata.parse(-1)
sheet_to_df_map = {}
sheet_to_df_map['T']=Tdata.parse(-1)
sheet_to_df_map['P']=Pdata.parse(-1)
sheet_to_df_map['I']=Idata.parse(-1)
E=['I','P','T']

#### Funcion para filtrar

In [12]:
def filtrar(lista:'En este caso la tabla con los productos vendidos de toda la semana como diccionario',
            empresa:'La sucursal en especifico en la cual se filtrara',
            posicion:'Indice para buscar',
            comparacion:'Producto que se buscara',
            lugar:'Lugar en donde se encuentra esa busqueda'):
    '''Esta funcion filtrara de la lista de productos vendidos de las sucursales, este sera un diccionario por lo cual buscara
    el producto que se le especifique'''
    if(comparacion in lista[empresa][lugar][posicion]):
        return posicion
    else:
        return -1
    

### Filtracion de un producto
Esto funciona para cualquier producto, solo faltaria extender en el parametro de comparacion, un map para todos los productos que se desean buscar

In [71]:
size=range(np.size(sheet_to_df_map['I']['DESCRIPCION']))
result=list(map(lambda e:list(map(lambda n:filtrar(sheet_to_df_map,e,n,'BUCHE','DESCRIPCION'),range(np.size(sheet_to_df_map[e]['DESCRIPCION'])))),['I','P','T']))
result=list(map(lambda n:np.array(result[n])>0,range(3)))
#x=list(map(lambda e,n:sheet_to_df_map[e].iloc[result[n],:],['I','P','T'],range(3)))
#Igual que en la linea anterior pero agregando una columna de zeros para la multiplicacion
y=list(map(lambda e,n:sheet_to_df_map[e].iloc[result[n],:].join(pd.DataFrame(np.ones(np.size(result[n])))),['I','P','T'],range(3)))


# Filtracion de todos los productos deseados que tambien necesitan de calcularse su consumo
Esta funcion es bajo la misma logica que la anterior con la diferencia de que filtra toda una lista de productos deseados

#### Funcion para elaborar arreglo filtrado

In [108]:
def array(P,E,sheet_to_df_map,search):
    result=list(map(lambda p:list(map(lambda e:list(map(lambda n:filtrar(sheet_to_df_map,e,n,p,search),range(np.size(sheet_to_df_map[e]['DESCRIPCION'])))),E)),P))
    result=list(map(lambda p:list(map(lambda n:np.array(result[p][n])>0,range(np.size(E)))),range(np.size(P))))
    y=list(map(lambda p:list(map(lambda e,n:sheet_to_df_map[e].iloc[result[p][n],:].join(pd.DataFrame(np.ones(np.size(result[p][n])))),E,range(np.size(E)))),range(np.size(P))))
    return y

# Calculo del consumo del insumo
Funciones de consumo de insumo

In [139]:
def consumo(producto:'Producto vendido',
            insumo:'Insumo a considerar del producto vendido',
            array:'arreglo vacio temporal para almacenar el consumo'):
    '''Esta funcion calcula segun el producto que se vendio la cantidad de producto de cierto insumo que se consume'''
    if("DORADITA VERDURAS ASADAS" in producto or "PANELA ASADA" in producto or "VERDURAS ASADAS" in producto):
        return array.append(0.0)
    else:
        if("DORADO PAPA" in producto):
            if("1/2 "+insumo in producto):
                return array.append(.03)
            else:
                return array.append(.06)
        elif("PITUFO" in producto):
            if("1/2 "+insumo in producto):
                return array.append(.03)
            else:
                return array.append(.06)
        elif("TORTA " in producto or "SUSHITORTA "in producto or"PAPA " in producto or "CHORIPAN " in producto or "GRINGA" in producto or "VAMPIRO" in producto or "GOBERNADOR GDE " in producto):
            if("1/2 "+insumo in producto):
                return array.append(.06)
            else:
                return array.append(.12)
        else:
            if("1/2 "+insumo in producto or "GOBERNADOR CH "+insumo in producto):
                return array.append(.03)
            else:
                return array.append(.06)  

In [101]:
def add_consumo(y:'La lista filtrada de los productos vendidos',
                n:'El consecutivo para identificar el tamaño de la lista de cada sucursal',
                m:'El consecutivo para identificar que insumo de la lista deseada se esta usando',
                array:'arreglo vacio temporal donde se guardara el consumo',
                i:'insumo',
                ):
    '''Esta funcion agrega a la tabla de productos filtrada la cantidad de cierto insumo consumida, la funcion que utiliza es para 
    lo que estaria considerado consumo de carnitas sera necesario diseñar consumos diferentes para otros productos que no sigan esta
    misma dinamica mas sin embargo todos los consumos se pueden agregar aqui'''
    list(map(lambda p:consumo(p,i,array),y[m][n]['DESCRIPCION']))
    y[m][n][0]=y[m][n][0]*array*y[m][n]['CANTIDAD']
    return array

## CARNES Y MARISCOS
Falta corregir detalle con pozoles

In [153]:
P=['BUCHE','PANCITA','LENGUA','PIERNA','ASADA','ARRACHERA','CHORIZO','MARLIN','CAMARON']
y=array(P,E,sheet_to_df_map,'DESCRIPCION')
# En este punto, el consumo de insumo ya esta considerado
z=list(map(lambda m,i:list(map(lambda n,a:add_consumo(y,n,m,a,i),range(3),[[],[],[]])),range(np.size(P)),P))
# Resultados
consumption=pd.DataFrame(list(map(lambda p:list(map(lambda n: np.round(np.sum(y[p][n][0]),2),range(np.size(E)))),range(np.size(P)))))
consumption.columns=E
consumption.index=P
consumption


,I,P,T
BUCHE,18.06,4.77,3.24
PANCITA,9.33,3.24,3.09
LENGUA,12.09,2.97,2.46
PIERNA,94.32,26.76,19.77
ASADA,20.82,13.38,6.21
ARRACHERA,12.18,7.44,3.30
CHORIZO,3.06,2.40,1.41
MARLIN,5.40,2.34,2.19
CAMARON,42.24,13.50,10.71


# Filtracion de productos donde su consumo es igual a la cantidad vendida
Para las cervezas no hay una distincion con las LATAS, estas son utilizadas en servicio a domicilio por lo cual cuando se compare contra los insumos surtidos estas deben ser consideradas como un total.

In [ ]:
def consumoB(y:'La lista filtrada de los productos vendidos',
             g:'El identificador para distinguir el insumo que se busca',
             e:'El identificador para distinguir la empresa en la que busco el insumo',
             p:'El identificador para distinguir contra que producto vendido se esta comparando el insumo'):
    '''Esta funcion es la encargada de realizar el calculo del insumo en las bebidas en donde solo si es GDE es doble el insumo'''
    if('GDE' in y[g][e]['DESCRIPCION'].iloc[p]):
        y[g][e][0]=y[g][e]['CANTIDAD']*2
    else:
        y[g][e][0]=y[g][e]['CANTIDAD']
    return 0

## REFRESCOS

In [168]:
P=['AGUA NATURAL','AGUA MINERAL','COCA COLA BOTELLA','COCA COLA SIN AZUCAR','COCA LIGHT BOTELLA','FANTA BOTELLA',
   'FRESCA BOTELLA','MANZANA LIFT BOTELLA','SPRITE BOTELLA','COCA COLA LATA','COCA LIGHT LATA','SPRITE LATA','SPRITE ZERO',
   'RUSA REFRESCO CH MINERAL','RUSA REFRESCO CH SPRITE','RUSA REFRESCO CH SPRITE ZERO','RUSA REFRESCO CH COCA','RUSA REFRESCO CH FRESCA',
   'RUSA REFRESCO GDE MINERAL','RUSA REFRESCO GDE SPRITE','RUSA REFRESCO GDE SPRITE ZERO','RUSA REFRESCO GDE COCA','RUSA REFRESCO GDE FRESCA']
y=array(P,E,sheet_to_df_map,'DESCRIPCION')
consumption=pd.DataFrame(list(map(lambda p:list(map(lambda n: np.round(np.sum(y[p][n]['CANTIDAD']),2),range(np.size(E)))),range(np.size(P)))))
consumption.columns=E
consumption.index=P
consumption.iloc[1,:]+=consumption.iloc[13,:]
consumption.iloc[8,:]+=consumption.iloc[14,:]
consumption.iloc[12,:]+=consumption.iloc[15,:]
consumption.iloc[2,:]+=consumption.iloc[16,:]
consumption.iloc[6,:]+=consumption.iloc[17,:]

consumption.iloc[1,:]+=consumption.iloc[18,:]*2
consumption.iloc[8,:]+=consumption.iloc[19,:]*2
consumption.iloc[12,:]+=consumption.iloc[20,:]*2
consumption.iloc[2,:]+=consumption.iloc[21,:]*2
consumption.iloc[6,:]+=consumption.iloc[22,:]*2

consumption.iloc[0:13,:]


,I,P,T
AGUA NATURAL,43,15,7
AGUA MINERAL,29,15,10
COCA COLA BOTELLA,205,81,41
COCA COLA SIN AZUCAR,9,2,3
COCA LIGHT BOTELLA,101,41,21
FANTA BOTELLA,10,6,5
FRESCA BOTELLA,8,5,1
MANZANA LIFT BOTELLA,20,9,0
SPRITE BOTELLA,21,13,10
COCA COLA LATA,14,6,5


## CERVEZAS

In [114]:
#P=['BEAGU','BEJUG','BEREF','BERLA','BECER','BEMIC','BEMIG','BEMCC','BEMCG','BERCC','BERCG','BERRC','BERRG']
P=['CORONA','CORONA LIGHT','MODELO ESPECIAL','NEGRA','PACIFICO','VICTORIA']
y=array(P,E,sheet_to_df_map,'DESCRIPCION')
z=list(map(lambda g:list(map(lambda e:list(map(lambda p:consumoB(y,g,e,p),range(np.size(y[g][e]['DESCRIPCION'])))),range(np.size(E)))),range(np.size(P))))
consumption=pd.DataFrame(list(map(lambda p:list(map(lambda n: np.round(np.sum(y[p][n][0]),2),range(np.size(E)))),range(np.size(P)))))
consumption.columns=E
consumption.index=P
consumption

,I,P,T
CORONA,398,62,50
CORONA LIGHT,106,13,36
MODELO ESPECIAL,28,14,12
NEGRA,58,22,7
PACIFICO,100,31,38
VICTORIA,69,9,13


## AGUAS

In [125]:
P=['CEBADA','COCO','AGUA DE GUAYABA','HORCHATA','JAMAICA','AGUA DE LIMON','TAMARINDO']
y=array(P,E,sheet_to_df_map,'DESCRIPCION')
consumption=pd.DataFrame(list(map(lambda p:list(map(lambda n: np.round(np.sum(y[p][n]['CANTIDAD']),2),range(np.size(E)))),range(np.size(P)))))
consumption.columns=E
consumption.index=P
consumption

,I,P,T
CEBADA,54,0,0
COCO,41,19,24
AGUA DE GUAYABA,64,24,21
HORCHATA,161,47,24
JAMAICA,81,34,24
AGUA DE LIMON,83,43,23
TAMARINDO,58,19,18


## JUGOS

In [129]:
P=['MANGO','BOING GUAYABA','FRESA']
y=array(P,E,sheet_to_df_map,'DESCRIPCION')
consumption=pd.DataFrame(list(map(lambda p:list(map(lambda n: np.round(np.sum(y[p][n]['CANTIDAD']),2),range(np.size(E)))),range(np.size(P)))))
consumption.columns=E
consumption.index=P
consumption

,I,P,T
MANGO,16,8,10
BOING GUAYABA,17,9,6
FRESA,5,7,2


## TORTAS Y PITUFOS

In [146]:
P=['TORTA','PITUFO','CHORIPAN','CHORIPAN PITUFO']
y=array(P,E,sheet_to_df_map,'DESCRIPCION')
consumption=pd.DataFrame(list(map(lambda p:list(map(lambda n: np.round(np.sum(y[p][n]['CANTIDAD']),2),range(np.size(E)))),range(np.size(P)))))
consumption.iloc[0,:]+=consumption.iloc[2,:]-consumption.iloc[3,:]
consumption.columns=E
consumption.index=['BOLILLO','BOLILLO MINI','CHORIPAN','CHORIPAN PITUFO']
consumption.iloc[0:2,:]

,I,P,T
BOLILLO,771,181,176
BOLILLO MINI,338,69,65


## TACOS DORADOS

In [147]:
P=['DORADO PAPA','DORADO FRIJOL','DORADO REQUESON']
y=array(P,E,sheet_to_df_map,'DESCRIPCION')
consumption=pd.DataFrame(list(map(lambda p:list(map(lambda n: np.round(np.sum(y[p][n]['CANTIDAD']),2),range(np.size(E)))),range(np.size(P)))))
consumption.columns=E
consumption.index=P
consumption

,I,P,T
DORADO PAPA,350,86,93
DORADO FRIJOL,303,64,58
DORADO REQUESON,184,48,35
